In [1]:
import pandas as pd
from statistics import median
import random
import csv
from projects import project_list
from collections import Counter
from batch_updation import *
from batching_algs import *

In [2]:
batch_total = 0
batch_duration = 0

In [3]:
def output_values(tr_results):
    l = []
    i = 0
    while i < len(tr_results):
        if tr_results[i] == 'passed':
            l.append(1)
        else:
            l.append(0)
        i += 1
    
    return l

In [4]:
def dynamic_batch(commits, print_flag =0):
    
    global linear_update, exp_update, random_linear, random_exp, freq_update, random_random
    global batchbisect, batchstop4, batchdivide4
    global batch_total
    
    
    rows = []
    batch_size = 16
    length = len(commits)
    builds = 0
    i = 0
    delays = []
    lines = 0
    
    algorithms = {'BATCHBISECT': batchbisect, 'BATCHSTOP4':batchstop4, 'BATCHDIVIDE4':batchdivide4}
    methods = {'linear': linear_update, 'exponential':exp_update, 'stagger':stagger_update, 
               'random_linear':random_linear, 'random_exponential':random_exp, 'random_random':random_random,
               }

    for m in methods:
        for alg in algorithms:
#             print('----------------------------------------------------------')
#             print('{} {}'.format(m, alg))
            batch_size = 16
            
            if m == 'linear':
                factors = [1, 2, 3, 4]
                
                for f in factors:
            
                    builds = 0
                    delays = []
                    i = 0
                    batch_size = 16
                    
#                     print('Factor {}'.format(f))
                    while i < length:
                        batch = commits[i:i+batch_size]
#                         print(batch_size, batch)

                        delays.extend([batch_size-x-1 for x in range(batch_size)])

                        batch_total = algorithms[alg](batch)
                        builds += batch_total
#                         print(alg, m)
#                         print(batch_size, batch, batch_total, builds)

                        i += batch_size
                        
                        if 0 in batch:
                            res = 0
                        else:
                            res = 1
                        #updating batch size
                        batch_size = methods[m](res, batch, batch_size, f)
                    
#                     print('project complete: builds {}'.format(builds))
                    saved = 100-(100*builds/length)
                    fr = 100*results.count(0)/length
                    rows.append([p_name, str(alg), '-1', str(m), f, builds, saved, length])
                    lines += 1
            
            elif (m == 'exponential') or (m == 'stagger'):
                factors = [2, 3, 4]
                
                for f in factors:
            
                    builds = 0
                    delays = []
                    i = 0
                    batch_size = 16
                    
#                     print('Factor {}'.format(f))
                    while i < length:
                        batch = commits[i:i+batch_size]
#                         print(batch_size, batch)

                        delays.extend([batch_size-x-1 for x in range(batch_size)])

                        batch_total = algorithms[alg](batch)
                        builds += batch_total
#                         print(alg, m)
#                         print(batch_size, batch, batch_total, builds)

                        i += batch_size
                        if 0 in batch:
                            res = 0
                        else:
                            res = 1

                        #updating batch size
                        batch_size = methods[m](res, batch, batch_size, f)
                    
#                     print('project complete: builds {}'.format(builds))
                    saved = 100-(100*builds/length)
                    fr = 100*results.count(0)/length
                    rows.append([p_name, str(alg), '-1', str(m), f, builds, saved, length])
                    lines += 1
            
            else:
                
                builds = 0
                delays = []
                i = 0
                batch_size = 16

                while i < length:
                    batch = commits[i:i+batch_size]
#                     print(batch_size, batch)

                    delays.extend([batch_size-x-1 for x in range(batch_size)])

                    batch_total = algorithms[alg](batch)
                    builds += batch_total
#                     print(alg, m)
#                     print(batch_size, batch, batch_total, builds)

                    i += batch_size
                    if 0 in batch:
                        res = 0
                    else:
                        res = 1

                    #updating batch size
                    batch_size = methods[m](res, batch, batch_size, -1)
                
#                 print('project complete: builds {}'.format(builds))
                saved = 100-(100*builds/length)
                fr = 100*results.count(0)/length
                rows.append([p_name, str(alg), '-1', str(m), -1, builds, saved, length])
                lines += 1
            
            #print([p_name, alg, '-1', m, saved, length])
    #print(lines)
    return rows
            

In [5]:
i = 0

standard_rows = []
ci_skip_rows = []
for p in project_list:
    p_name = p.split('/')[1]
    
    if (p_name == 'BuildCraft-BuildCraft.csv') | (p_name == 'RS485-LogisticsPipes.csv') | (p_name == 'datastax-java-driver.csv') :
        continue
        
    data = pd.read_csv('../dynamic batching/ci_skip_data/extracted_project_travis/' + p_name)
    
    branch_type = data['git_branch'].tolist()
    if 'master' in branch_type:
        data = data[ data['git_branch'] == 'master']
    else:
        data = data[ data['git_branch'] == 'trunk']
        
    total_length = len(data) - 100
    run_data = data.iloc[100:]
    
    if len(run_data) == 0:
        print(i)
        i += 1
        continue
    results = output_values(run_data['tr_status'].tolist())
    duration = run_data['tr_duration'].tolist()
    
    print(p_name)
    rows = dynamic_batch(results)
    standard_rows.extend(rows)
    
    
    run_data = data[ data.verdict == 0]
    run_data = run_data.iloc[100:]
    if len(run_data) == 0:
        print(i)
        i += 1
        continue
    results = output_values(run_data['tr_status'].tolist())
    rows = dynamic_batch(results, print_flag=1)
    ci_skip_rows.extend(rows)
    
    

junit-team-junit5.csv
ratpack-ratpack.csv
p6spy-p6spy.csv
apache-tajo.csv
osmdroid-osmdroid.csv
DSpace-DSpace.csv
apache-nifi.csv
caelum-vraptor4.csv
grails-grails-core.csv
square-wire.csv
puniverse-quasar.csv
dropwizard-dropwizard.csv
essentials-Essentials.csv
mybatis-mybatis-3.csv
naver-pinpoint.csv
FasterXML-jackson-databind.csv
samtools-htsjdk.csv
deeplearning4j-deeplearning4j.csv
open-keychain-open-keychain.csv
yegor256-takes.csv
keycloak-keycloak.csv
rackerlabs-blueflood.csv
xetorthio-jedis.csv
orbeon-orbeon-forms.csv
linkedin-gobblin.csv
JabRef-jabref.csv
jOOQ-jOOQ.csv
structr-structr.csv
floragunncom-search-guard.csv
SpongePowered-SpongeForge.csv
sakaiproject-sakai.csv
dreamhead-moco.csv
nutzam-nutz.csv
CloudSlang-cloud-slang.csv
geotools-geotools.csv
CloudifySource-cloudify.csv
SpongePowered-SpongeCommon.csv
apache-zeppelin.csv
typetools-checker-framework.csv
geoserver-geoserver.csv
ngageoint-geowave.csv
sparklemotion-nokogiri.csv
Graylog2-graylog2-server.csv
cloudfoundry-uaa.

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (61) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


apache-pdfbox.csv
apache-sling.csv
SonarSource-sonar-java.csv


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,8,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


killbill-killbill.csv


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,8,20,51,53,61) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


checkstyle-checkstyle.csv


In [6]:
df = pd.DataFrame(standard_rows, columns=['project', 'algorithm', 'batch_size', 'update_method', 'factor', 'num_of_builds', 'builds_saved', 'testall_size'])
df.to_csv('standard_dynamic_batching.csv')

In [7]:
df = pd.DataFrame(ci_skip_rows, columns=['project', 'algorithm', 'batch_size', 'update_method', 'factor', 'num_of_builds', 'builds_saved', 'testall_size'])
df.to_csv('ci_skipped_dynamic_batching.csv')